Interactive plots

In [7]:
from IPython.core.display import display
%reload_ext autoreload
%autoreload 2
import ipywidgets as widgets
import matplotlib.pyplot as plt
import itertools
import pandas as pd
from ipywidgets import interactive, interact, interact_manual, interactive_output
from plotly.offline import init_notebook_mode,iplot
import new
import FileHandling
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set_context('notebook')
sns.set_style('white')
init_notebook_mode(connected=True)
import numpy as np
subjects = range(201, 212)
targets = range(8)
envs = ['U', 'W']
poss = ['W','S']
blocks = range(5)

In [6]:
# button = widgets.Button(description='Update')
# df = px.data.gapminder()
@interact_manual
def scatter_plot(subject=subjects,target=targets,env=envs,pos=poss,block=blocks):
    eye,holo,imu = new.one_trial(subject, target, env, pos, block)
    eye, holo, imu, peaks, filtered_x,filtered_y,target_vertical,target_horizontal= new.data_prep(eye,holo,imu)
    # fig ,axs= plt.subplots(1,2,figsize=(12,6))
    # axs[0].scatter(filtered_x,filtered_y)
    # axs[1].scatter(target_horizontal,target_vertical)
    # plt.show()
#     fig = px.scatter(data_frame=eye,x='filtered_x',y='filtered_y',color='confidence',animation_frame='timestamp',range_x=[0.4,0.5],range_y=[0.2,0.3])
    fig = px.scatter(data_frame=eye,x='target_horizontal',y='target_vertical',color='confidence',animation_frame='timestamp')
    fig.show()
    # iplot(fig)
scatter_plot


interactive(children=(Dropdown(description='subject', options=(201, 202, 203, 204, 205, 206, 207, 208, 209, 21…

<function __main__.scatter_plot(subject=range(201, 212), target=range(0, 8), env=['U', 'W'], pos=['W'], block=range(0, 5))>

In [8]:
def Pickling():
    for subject in subjects:
        [imu_file_list, eye_file_list, hololens_file_list] = FileHandling.get_one_subject_files(subject, refined=True)
        for target, env, pos, block in itertools.product(targets, envs, poss, blocks):
            current_info = [target, env, pos, block]
            try:
                # eye = FileHandling.file_as_pandas(FileHandling.get_file_by_info(eye_file_list, current_info),
                #                                   refined=True)
                holo = FileHandling.file_as_pandas(FileHandling.get_file_by_info(hololens_file_list, current_info))
                # imu = FileHandling.file_as_pandas(FileHandling.get_file_by_info(imu_file_list, current_info))
                #
                path='/Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/'
                holo.to_pickle(path=path +'holo_'+str(subject)+'_'+ FileHandling.make_trial_info(current_info)+'.pkl')
                print('saved',path +'holo_'+str(subject)+'_'+ FileHandling.make_trial_info(current_info)+'.pkl' )
                # if eye.shape[0] < 100: print('empty eye data');continue
                # data_analysis(eye, holo, imu)
            except ValueError as err:
                print(err, current_info)


saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PW_B0.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PW_B1.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PW_B2.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PW_B3.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PW_B4.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PS_B0.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PS_B1.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pickled_data/holo_201_T0_EU_PS_B2.pkl
saved /Users/shin-yonghwan/Documents/GitHub/HeadEyeTracking/Datasets/2ndData/Pic